In [11]:
import tribo as tb
import numpy as np
import os 
import re
import pandas as pd

def split_string(s):
    return re.split('[-_]', s)

class Tribo_file:
	def __init__(self,file_folder, file_name):
		self.file_name = file_name
		self.file_folder = file_folder
		self.percent,self.name,self.force,self.speed,self.test = split_string(self.file_name)
		
		self.load_data()

	def load_data(self):
		self.data,self.xpos,self.Fx,self.Fz = tb.get_data(self.file_folder+"\\"+self.file_name,"all")
		

	def process_data(self,cutoff=0.1):

		self.xpos_peaks, self.xpos_valleys = tb.find_x_segments(self.xpos)

		segments=tb.segment_data(self.xpos, self.xpos_peaks, self.xpos_valleys,self.Fx,self.Fz)

		self.upward_sections, self.forward_friction, self.downward_sections, self.backward_friction=segments

		self.CoF_array = tb.calculate_CoF(self.upward_sections, self.forward_friction, self.downward_sections, self.backward_friction,cutoff)

		self.CoF_avg=np.average(self.CoF_array)
		self.CoF_std=np.std(self.CoF_array)

		pad_length = 3000 - len(self.CoF_array)
		if pad_length > 0:
			padded_array = np.pad(file_n.CoF_array, (0, pad_length), 'constant', constant_values=self.CoF_avg)
		else:
			padded_array = file_n.CoF_array

		self.CoF_array=padded_array




		

folder = 'DATA'

files_in_folder = os.listdir(folder)
print(f" Found {len(files_in_folder)} files in {folder}")

Tribo_files = []
complete_CoF_df = pd.DataFrame()

for file_name in files_in_folder:
	file_n=Tribo_file(folder,file_name)
	file_n.process_data(0.2)
	Tribo_files.append(file_n)
	# print(f"Processed {file_name} with CoF {file_n.CoF_array.shape} and % {file_n.percent} ")
	complete_CoF_df[file_name]=file_n.CoF_array

speeds=[]
names=[]

for file in Tribo_files:
	if file.speed not in speeds:
		speeds.append(file.speed)

	if file.name not in names:
		names.append(file.name)

# print(f" Procesing {speeds}")

speed_sheets=[]

# for speed in speeds:
# 	files_with_speed = [file for file in Tribo_files if file.speed == speed]

	
# 	rows=[]
# 	ii=0
# 	for file in files_with_speed:
# 		data_to_append = [ file.name,  file.CoF_avg,  file.CoF_std,file.percent,file.force,file.test]
# 		rows.append(data_to_append)

# 	df = pd.DataFrame(rows,columns=['Name','CoF_avg','CoF_std','Percent','Force','Test'])


# 	speed_sheets.append(df)

rows=[]
for file in Tribo_files:
	data_to_append = [ file.name,  file.CoF_avg,  file.CoF_std,file.speed, file.percent, file.force,file.test]
	rows.append(data_to_append)
	

df = pd.DataFrame(rows,columns=['Name','CoF_avg','CoF_std','Speed','Percent','Force','Test'])

print("This is the complete df\n",df)
for speed in speeds:
	speed_sheet=df[df['Speed']==speed]
	print("speed sheet\n",speed_sheet)
	speed_sheets.append(speed_sheet)

rows_total=[]
total_dfs=[]
with pd.ExcelWriter('output.xlsx') as writer: 
	for df,speed  in zip(speed_sheets,speeds): 
		rows_total=[]

		for name in df['Name'].unique():
			for force in df['Force'].unique():
				print(f"----------{name}\t{force}------------")
				df2=df[(df['Name']==name) & (df['Force']==force)]
				CoF_total_avg=np.average(df2['CoF_avg'])
				CoF_total_std=np.average(df2['CoF_std'])
				row=[name,CoF_total_avg,CoF_total_std,speed,df2.get("Force").iloc[0],df2.get("Percent").iloc[0]]
				
			
				print(row)
				rows_total.append(row)
		df=pd.DataFrame(rows_total,columns=['Name','CoF_avg','CoF_std','Speed', 'Force','Percent'])
		total_dfs.append(df)
		df.to_excel(writer, sheet_name=speed, index=False)
	
	complete_df=pd.concat(total_dfs)
	complete_df.to_excel(writer, sheet_name='Total', index=False)
	complete_CoF_df.to_excel(writer, sheet_name='CoF', index=True)
	
		



 Found 25 files in DATA
This is the complete df
     Name   CoF_avg   CoF_std   Speed Percent Force    Test
0    OA0  0.149389  0.017284  100mms       0   10N  test12
1    OA0  0.129268  0.020919  100mms       0   10N   test3
2    OA0  0.110910  0.016482  100mms       0   10N   test8
3    OA0  0.157980  0.018407   20mms       0   10N  test14
4    OA0  0.178555  0.020883   20mms       0   10N   test4
5    OA0  0.150612  0.018965   20mms       0   10N   test6
6    OA0  0.146705  0.007325  100mms       0   20N  test13
7    OA0  0.131942  0.009320  100mms       0   20N   test2
8    OA0  0.144244  0.009370  100mms       0   20N   test7
9    OA0  0.158089  0.009677   20mms       0   20N   test1
10   OA0  0.170787  0.010615   20mms       0   20N  test11
11   OA0  0.150995  0.011565   20mms       0   20N   test5
12  ZADP  0.121985  0.006986  100mms       1   20N   Test4
13  ZADP  0.112659  0.006922  100mms       1   20N   Test5
14  ZADP  0.111450  0.008739  100mms       1   20N   Test6
15  ZAD

C:\Users\cordo\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\cordo\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [13]:
(df.get("Force").iloc[0])

'20N'